<a href="https://colab.research.google.com/github/amem0004/alisha/blob/main/ADS2002_Alisha.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Allows user to load in files

!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
driver = GoogleDrive(gauth)
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
# import libraries 
import pandas as pd # Import Pandas for data manipulation using dataframes
import numpy as np # Import Numpy for data statistical analysis 
import matplotlib.pyplot as plt # Import matplotlib for data visualisation
import random
import seaborn as sns
from fbprophet import Prophet

In [4]:
# Data converter
from datetime import datetime
from numpy import distutils
import distutils

# Converts the contents in a .tsf file into a dataframe and returns it along with other meta-data of the dataset: frequency, horizon, whether the dataset contains missing values and whether the series have equal lengths
#
# Parameters
# full_file_path_and_name - complete .tsf file path
# replace_missing_vals_with - a term to indicate the missing values in series in the returning dataframe
# value_column_name - Any name that is preferred to have as the name of the column containing series values in the returning dataframe
def convert_tsf_to_dataframe(full_file_path_and_name, replace_missing_vals_with = 'NaN', value_column_name = "series_value"):
    col_names = []
    col_types = []
    all_data = {}
    line_count = 0
    frequency = None
    forecast_horizon = None
    contain_missing_values = None
    contain_equal_length = None
    found_data_tag = False
    found_data_section = False
    started_reading_data_section = False

    with open(full_file_path_and_name, 'r', encoding='cp1252') as file:
        for line in file:
            # Strip white space from start/end of line
            line = line.strip()

            if line:
                if line.startswith("@"): # Read meta-data
                    if not line.startswith("@data"):
                        line_content = line.split(" ")
                        if line.startswith("@attribute"):
                            if (len(line_content) != 3):  # Attributes have both name and type
                                raise Exception("Invalid meta-data specification.")

                            col_names.append(line_content[1])
                            col_types.append(line_content[2])
                        else:
                          if len(line_content) != 2:  # Other meta-data have only values
                                raise Exception("Invalid meta-data specification.")

                          if line.startswith("@frequency"):
                                frequency = line_content[1]
                          elif line.startswith("@horizon"):
                                forecast_horizon = int(line_content[1])
                          elif line.startswith("@missing"):
                                contain_missing_values = bool(distutils.util.strtobool(line_content[1]))
                          elif line.startswith("@equallength"):
                                contain_equal_length = bool(distutils.util.strtobool(line_content[1]))

                    else:
                        if len(col_names) == 0:
                            raise Exception("Missing attribute section. Attribute section must come before data.")

                        found_data_tag = True
                elif not line.startswith("#"):
                    if len(col_names) == 0:
                        raise Exception("Missing attribute section. Attribute section must come before data.")
                    elif not found_data_tag:
                        raise Exception("Missing @data tag.")
                    else:
                        if not started_reading_data_section:
                            started_reading_data_section = True
                            found_data_section = True
                            all_series = []

                            for col in col_names:
                                all_data[col] = []

                        full_info = line.split(":")

                        if len(full_info) != (len(col_names) + 1):
                            raise Exception("Missing attributes/values in series.")

                        series = full_info[len(full_info) - 1]
                        series = series.split(",")

                        if(len(series) == 0):
                            raise Exception("A given series should contains a set of comma separated numeric values. At least one numeric value should be there in a series. Missing values should be indicated with ? symbol")

                        numeric_series = []

                        for val in series:
                            if val == "?":
                              numeric_series.append(replace_missing_vals_with)
                            else:
                                numeric_series.append(float(val))

                        if (numeric_series.count(replace_missing_vals_with) == len(numeric_series)):
                            raise Exception("All series values are missing. A given series should contains a set of comma separated numeric values. At least one numeric value should be there in a series.")

                        all_series.append(pd.Series(numeric_series).array)

                        for i in range(len(col_names)):
                            att_val = None
                            if col_types[i] == "numeric":
                                att_val = int(full_info[i])
                            elif col_types[i] == "string":
                                att_val = str(full_info[i])
                            elif col_types[i] == "date":
                                att_val = datetime.strptime(full_info[i], '%Y-%m-%d %H-%M-%S')
                            else:
                                raise Exception("Invalid attribute type.") # Currently, the code supports only numeric, string and date types. Extend this as required.
                            if(att_val == None):
                                raise Exception("Invalid attribute value.")
                            else:
                                all_data[col_names[i]].append(att_val)

                line_count = line_count + 1

        if line_count == 0:
            raise Exception("Empty file.")
        if len(col_names) == 0:
            raise Exception("Missing attribute section.")
        if not found_data_section:
            raise Exception("Missing series information under data section.")

        all_data[value_column_name] = all_series
        loaded_data = pd.DataFrame(all_data)

        return loaded_data, frequency, forecast_horizon, contain_missing_values, contain_equal_length


# Example of usage
# loaded_data, frequency, forecast_horizon, contain_missing_values, contain_equal_length = convert_tsf_to_dataframe("TSForecasting/tsf_data/sample.tsf")

# print(loaded_data)
# print(frequency)
# print(forecast_horizon)
# print(contain_missing_values)
# print(contain_equal_length)

In [7]:
df = convert_tsf_to_dataframe('gdrive/My Drive/ADS2002 - Solar Farm/phase_1_data.tsf')
# Alternative and easier way
df1 = pd.read_csv('gdrive/My Drive/ADS2002 - Solar Farm/solar_data.csv', parse_dates=['start_timestamp'], index_col=['start_timestamp'])
# Note: this was data frame was created using R and Nan values were removed
df1 = df1.drop(['Unnamed: 0'], axis = 1)
df1 = df1.rename(columns={"start_timestamp": "Timestamp", "series_value": "Value", "series_name": "Name"})
#df1= df1.set_index('Timestamp'
df1

,Name,Value
start_timestamp,,
2016-07-03 21:30:00,Building0,283.80
2016-07-03 21:45:00,Building0,283.80
2016-07-03 22:00:00,Building0,283.80
2016-07-03 22:15:00,Building0,606.00
2016-07-03 22:30:00,Building0,606.00
...,...,...
2020-09-30 22:45:00,Solar5,25.28
2020-09-30 23:00:00,Solar5,25.30
2020-09-30 23:15:00,Solar5,28.62


**SPLITTING DATA FURTHER** 

In [8]:
df_Building0 = df1.loc[df1['Name'] == 'Building0']
df_Building1 = df1.loc[df1['Name'] == 'Building1']
df_Building3 = df1.loc[df1['Name'] == 'Building3']
df_Building4 = df1.loc[df1['Name'] == 'Building4']
df_Building5 = df1.loc[df1['Name'] == 'Building5']
df_Building6 = df1.loc[df1['Name'] == 'Building6']
df_Solar0 = df1.loc[df1['Name'] == 'Solar0']
df_Solar1 = df1.loc[df1['Name'] == 'Solar1']
df_Solar2 = df1.loc[df1['Name'] == 'Solar2']
df_Solar3 = df1.loc[df1['Name'] == 'Solar3']
df_Solar4 = df1.loc[df1['Name'] == 'Solar4']
df_Solar5 = df1.loc[df1['Name'] == 'Solar5']

In [9]:
df_Building0

,Name,Value
start_timestamp,,
2016-07-03 21:30:00,Building0,283.8
2016-07-03 21:45:00,Building0,283.8
2016-07-03 22:00:00,Building0,283.8
2016-07-03 22:15:00,Building0,606.0
2016-07-03 22:30:00,Building0,606.0
...,...,...
2020-09-30 22:45:00,Building0,96.9
2020-09-30 23:00:00,Building0,96.9
2020-09-30 23:15:00,Building0,37.4


Example of splitting the time frames to hours, minute

In [10]:
example = df_Building0[df_Building0.index.hour == 12]
example

,Name,Value
start_timestamp,,
2016-07-04 12:00:00,Building0,458.0
2016-07-04 12:15:00,Building0,444.3
2016-07-04 12:30:00,Building0,444.3
2016-07-04 12:45:00,Building0,444.3
2016-07-05 12:00:00,Building0,455.0
...,...,...
2020-09-29 12:45:00,Building0,149.6
2020-09-30 12:00:00,Building0,162.4
2020-09-30 12:15:00,Building0,159.9
